# [1731. The Number of Employees Which Report to Each Employee](https://leetcode.com/problems/the-number-of-employees-which-report-to-each-employee/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Employees

<pre>+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+-------------+----------+</pre>
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null).


For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.



Example 1:

Input:
Employees table:
<pre>+-------------+---------+------------+-----+
| employee_id | name    | reports_to | age |
+-------------+---------+------------+-----+
| 9           | Hercy   | null       | 43  |
| 6           | Alice   | 9          | 41  |
| 4           | Bob     | 9          | 36  |
| 2           | Winston | null       | 37  |
+-------------+---------+------------+-----+</pre>
Output:
<pre>+-------------+-------+---------------+-------------+
| employee_id | name  | reports_count | average_age |
+-------------+-------+---------------+-------------+
| 9           | Hercy | 2             | 39          |
+-------------+-------+---------------+-------------+</pre>
Explanation: Hercy has 2 people report directly to him, Alice and Bob. Their average age is (41+36)/2 = 38.5, which is 39 after rounding it to the nearest integer.

In [13]:
# Pandas schema

import pandas as pd

data = [[9, 'Hercy', None, 43], [6, 'Alice', 9, 41], [4, 'Bob', 9, 36], [2, 'Winston', None, 37]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'reports_to', 'age']).astype(
    {'employee_id': 'Int64', 'name': 'object', 'reports_to': 'Int64', 'age': 'Int64'})

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# To pyspark schema

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("employee_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("reports_to", IntegerType(), True),
    StructField("age", IntegerType(), True)
])

employees_df = spark.createDataFrame(employees)
employees_df.show()

+-----------+-------+----------+---+
|employee_id|   name|reports_to|age|
+-----------+-------+----------+---+
|          9|  Hercy|       NaN| 43|
|          6|  Alice|       9.0| 41|
|          4|    Bob|       9.0| 36|
|          2|Winston|       NaN| 37|
+-----------+-------+----------+---+



In [2]:
employees

,employee_id,name,reports_to,age
0,9,Hercy,<NA>,43
1,6,Alice,9,41
2,4,Bob,9,36
3,2,Winston,<NA>,37


In [16]:
# In spark dataframe
employees_df.alias('e1')\
            .join(employees_df.alias('e2'),
                  col('e1.employee_id')==col('e2.reports_to'),
                  'inner')\
            .groupby('e1.employee_id','e1.name').agg(count('e2.employee_id').alias('reports_count'),round(avg('e2.age'),0).alias('average_age'))\
            .show()



+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|       39.0|
+-----------+-----+-------------+-----------+



In [ ]:
# In spark SQL

employees_df.createOrReplaceTempView('employees')

spark.sql('''

select e1.employee_id,
       e1.name,
       count(e2.employee_id) as reports_count,
       round(avg(e2.age)) as  average_age
from
employees e1 join employees e2
on e1.employee_id=e2.reports_to
group by e1.employee_id,e1.name
order by e1.employee_id

''').show()

+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|         39|
+-----------+-----+-------------+-----------+


In [ ]:
spark.stop()